In [1]:
%pylab inline
from enum import Enum

Populating the interactive namespace from numpy and matplotlib


# Algebra Solver

Solve `1/2*10*(5+2)/3`.

We should follow PEMDAS:

1. Parenthesis
2. Exponents
3. Multiplication/Division
4. Addition/Subtraction

In [2]:
class OpCode(Enum):
    ADD = 1
    SUBTRACT = 2
    MULTIPLY = 3
    DIVIDE = 4

In [3]:
class Token():
    def __init__(self, tokenType, value):
        self.tokenType = tokenType
        self.value = value
    def __repr__(self):
        return "[%s](%s)" % (self.tokenType, self.value)

In [4]:
class Operation():
    def __init__(self, left, right, opcode):
        self.left = left
        self.right = right
        self.opcode = opcode
    def __repr__(self):
        return '%s %s %s' % (self.left, self.opcode, self.right)
    def result(self):
        if self.opcode == OpCode.ADD:
            return self.left.result() + self.right.result()
        elif self.opcode == OpCode.SUBTRACT:
            return self.left.result() - self.right.result()
        elif self.opcode == OpCode.MULTIPLY:
            return self.left.result() * self.right.result()
        elif self.opcode == OpCode.DIVIDE:
            return sel.fleft.result() / self.right.result()
        
class Constant():
    def __init__(self, value):
        self.value = value
    def result(self):
        return self.value

In [5]:
five = Constant(5)
six = Constant(6)
add = Operation(five, six, OpCode.MULTIPLY)
add.result()

30

In [6]:
class Parser():
    def __init__(self, expression):
        print('Expression=%s' % expression)
        print('Length=%i' % len(expression))
        self.expression = expression
        self.index = 0
        self.expressionLength = len(self.expression)
        self.tokens = []
        # Start
        self.tokenize()
        print(self.tokens)
        
    def tokenize(self):
        while(not self.end()):
            print('current index %i and char %s' % (self.index, self.nextChar()))
            self.eatWhitespace()
            token = self.nextToken()
            self.tokens.append(token)
        
    def end(self):
        if self.index >= self.expressionLength: return True
        return False
    
    def incr(self):
        self.index += 1
    
    def nextToken(self):
        if self.nextChar() == '+': 
            self.incr()
            return Token('OP', 'ADD')
        elif self.nextChar() == '-':
            self.incr()
            return Token('OP', 'SUB')
        elif self.nextChar() == '*':
            self.incr()
            return Token('OP', 'MUL')
        elif self.nextChar() == '/':
            self.incr()
            return Token('OP', 'DIV')
        elif self.nextChar().isdigit():
            return Token('C', self.nextDigit())
            
    def nextChar(self):
        return self.expression[self.index:self.index+1]
    
    def nextDigit(self):
        digit = ''
        while self.nextChar().isdigit():
            digit += self.nextChar()
            self.index += 1
        return digit
    
    def eatWhitespace(self):
        while(self.expression[self.index:self.index+1] == ' '):
            self.index += 1

In [7]:
parser = Parser('30*2+1124/5')

Expression=30*2+1124/5
Length=11
current index 0 and char 3
current index 2 and char *
current index 3 and char 2
current index 4 and char +
current index 5 and char 1
current index 9 and char /
current index 10 and char 5
[[C](30), [OP](MUL), [C](2), [OP](ADD), [C](1124), [OP](DIV), [C](5)]


In [8]:
class Stack():
    def __init__(self):
        self.items = []
        
    def __repr__(self):
        return '%s' % self.items
    
    def push(self, item):
        self.items.append(item)
        
    def pop(self):
        item = self.items[-1]
        del self.items[-1]
        return item

In [9]:
class ExpressionTree():
    def __init__(self, tokens):
        self.tokens = tokens
        self.intermediateTokens = []
        self.index = 0
        self.firstPass()
        print(self.intermediateTokens)
        self.tokens = self.intermediateTokens
        self.intermediateTokens = []
        self.secondPass()
        print(self.intermediateTokens)
    
    def end(self):
        if self.index >= len(self.tokens): return True
        else: return False
    
    def nextToken(self,skip=0):
        return self.tokens[self.index+skip:self.index+1+skip][0]
    
    def previousToken(self):
        return self.tokens[self.index-1:self.index][0]
    
    def incr(self, skip=1):
        self.index+=skip
        
    """ First do multiplication and division """
    def firstPass(self):
        while(not self.end()):
            if self.nextToken().tokenType == 'C':
                self.incr()
            elif self.nextToken().tokenType == 'OP' and self.nextToken().value == 'MUL':
                operation = Operation(self.previousToken(), self.nextToken(1), OpCode.MULTIPLY)
                token = Token('IOP', operation)
                self.intermediateTokens.append(token)
                self.incr(2)
            elif self.nextToken().tokenType == 'OP' and self.nextToken().value == 'DIV':
                operation = Operation(self.previousToken(), self.nextToken(1), OpCode.DIVIDE)
                token = Token('IOP', operation)
                self.intermediateTokens.append(token)
                self.incr(2)
            else:
                self.intermediateTokens.append(self.nextToken())
                self.incr()
            
            
    """ Next do addition and subtraction"""
    def secondPass(self):
        while(not self.end()):
            if self.nextToken().tokenType == 'IOP':
                self.incr()
            elif self.nextToken().tokenType == 'OP' and self.nextToken().value == 'ADD':
                operation = Operation(self.previousToken().value, self.nextToken().value, OpCode.ADD)
                token = Token('IOP', operation)
                self.intermediateTokens.append(token)
                self.incr(2)
            else:
                self.intermediateTokens.append(self.nextToken())
                self.incr()

In [10]:
et = ExpressionTree(parser.tokens)

[[IOP]([C](30) OpCode.MULTIPLY [C](2)), [OP](ADD), [IOP]([C](1124) OpCode.DIVIDE [C](5))]
[]


In [11]:
et.intermediateTokens

[]

In [12]:
op = Operation(et.intermediateTokens[0], et.intermediateTokens[2], OpCode.ADD)
op.result()

IndexError: list index out of range

In [20]:
symbols = {'+':'add', '-':'subtract', '*':'multiply', '/':'divide', '^':'power', '(':'left-parenthesis', ')':'right-parenthesis'}
symbols

{'(': 'left-parenthesis',
 ')': 'right-parenthesis',
 '*': 'multiply',
 '+': 'add',
 '-': 'subtract',
 '/': 'divide',
 '^': 'power'}